1. Suppose you have a daily univariate time series, and you want to forecast the next seven days. Which RNN architecture should you use, i.e. how many neurons you plan to set up?

An RNN architecture it is a good option since we have univariate time series. However, there may be a problem with sort-term memory. We can solve that by using Long Short-Term memory (LSTM) architecture. LSTM is useful with time series data. LSTM will recognize significant inputs and preserve it as long as it's needed. <br/>
Another option can be Gated Recurrent Unit architecture which is similar to LSTM. It solves the same problem in a more simple way. <br/>
We need to forecast the next seven days. So, we will need seven neurons in the output layer. 

2. Why do people use Encoder–Decoder RNNs rather than plain sequence-to-sequence RNNs for automatic translation?

Encoder-Decoder RNN converts the input data into a context semantic vector, which is the word embedding. It is a form of preprocessing the data which makes the data understandable to the machine. <br/>
After the encoder has created all the context vectors the decoder decodes the context into an output sequence. <br/>
In the case of automatic translation the encoder will convert the sentence into word embeddings. Then the decoder from decodes these vectors into an output in the chosen language.

3. (optional) Install the tensorflow_addons (pip install tensorflow-addons) and test the Python script of automatic translation (RNN_demo5.ipynb). 

In [26]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

In [27]:
# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"


In [28]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

In [29]:
# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

In [30]:
tf.config.list_physical_devices('GPU')

[]

In [31]:
if not tf.test.is_gpu_available():
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")


No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


In [32]:
# Common imports
import numpy as np
import os


In [33]:
# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

In [34]:
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

ModuleNotFoundError: No module named 'matplotlib'

In [12]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [13]:
tf.random.set_seed(42)

In [14]:
vocab_size = 100
embed_size = 10

In [15]:
import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                 output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba])

In [16]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")


In [17]:
X = np.random.randint(100, size=10*1000).reshape(1000, 10)
Y = np.random.randint(100, size=15*1000).reshape(1000, 15)
X_decoder = np.c_[np.zeros((1000, 1)), Y[:, :-1]]
seq_lengths = np.full([1000], 15)

history = model.fit([X, X_decoder, seq_lengths], Y, epochs=2)

Epoch 1/2
32/32 [==============================] - 4s 72ms/step - loss: 4.6053
Epoch 2/2
32/32 [==============================] - 2s 72ms/step - loss: 4.6036
